In [1]:
!pip install ultralytics --quiet
!pip install -U ipywidgets --quiet
!pip uninstall wandb -y

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.2 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.9.2 which is incompatible.
Found existing installation: wandb 0.18.3
Uninstalling wandb-0.18.3:
  Successfully uninstalled wandb-0.18.3


In [2]:
import os
import shutil
from distutils.dir_util import copy_tree

from ultralytics.data.utils import autosplit
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# HYPERPARAMETERS
TRAIN_FRACTION = 0.9
VAL_FRACTION = 0.05
TEST_FRACTION = 0.05
EPOCHS = 100

assert ((TRAIN_FRACTION + VAL_FRACTION + TEST_FRACTION) == 1.0), "Sum of split fraction != 1"

In [4]:
# Copy train images from kaggle input directory to kaggle output directory
dataset_source_dir = '/kaggle/input/fixed-wing-uavs/dataset-07-10-2024/'
dataset_target_dir = '/kaggle/working/dataset/'

data_yaml_source_path = '/kaggle/working/dataset/data_kaggle.yaml'
data_yaml_target_path = '/kaggle/working/data_kaggle.yaml'

os.makedirs(dataset_target_dir, exist_ok=True)
copy_tree(dataset_source_dir, dataset_target_dir)
shutil.move(data_yaml_source_path, data_yaml_target_path)

'/kaggle/working/data_kaggle.yaml'

In [5]:
# create data split
autosplit(path="/kaggle/working/dataset", weights = (TRAIN_FRACTION, VAL_FRACTION, TEST_FRACTION))

Autosplitting images from /kaggle/working/dataset


100%|██████████| 891/891 [00:00<00:00, 19336.29it/s]


In [6]:
# use pre-trained yolov11 model
model = YOLO("yolo11n.pt")
results = model.train(data="/kaggle/working/data_kaggle.yaml",
                      epochs=EPOCHS,
                      device=[0, 1],
                      name='custom_model')

Ultralytics 8.3.9 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
                                                CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/kaggle/working/data_kaggle.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1], workers=8, project=None, name=custom_model3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt

train: Scanning /kaggle/working/dataset... 659 images, 141 backgrounds, 2 corrupt: 100%|██████████| 802/802 [00:00<00:00, 901.83it/s] 


train: WARNING ⚠️ /kaggle/working/dataset/517.jpg: ignoring corrupt image/label: cannot identify image file '/kaggle/working/dataset/517.jpg'
train: WARNING ⚠️ /kaggle/working/dataset/8.jpg: ignoring corrupt image/label: cannot identify image file '/kaggle/working/dataset/8.jpg'
train: New cache created: /kaggle/working/dataset.cache


/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/dataset... 43 images, 6 backgrounds, 0 corrupt: 100%|██████████| 49/49 [00:00<00:00, 1025.02it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
val: New cache created: /kaggle/working/dataset.cache


...

                   all         49         48      0.941      0.833      0.898      0.759

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      1.26G     0.3574     0.3155     0.9228          5        640: 100%|██████████| 50/50 [00:09<00:00,  5.15it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00, 11.97it/s]


                   all         49         48      0.912      0.864      0.899      0.773

100 epochs completed in 0.319 hours.
Optimizer stripped from runs/detect/custom_model3/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/custom_model3/weights/best.pt, 5.5MB

Validating runs/detect/custom_model3/weights/best.pt...
Ultralytics 8.3.9 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
                                                CUDA:1 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  6.26it/s]


                   all         49         48      0.953      0.854      0.921      0.782
Speed: 0.2ms preprocess, 4.3ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/custom_model3


In [33]:
import random

def generate_random_image_paths(num_paths):
    with open("/kaggle/working/autosplit_test.txt", "r") as f:
        data = f.read().replace('./', '/kaggle/working/').split('\n')
        data = data[:-1]
    
    generated_paths = []
    image_nums = random.sample(range(0, len(data)), num_paths)

    for i in image_nums:
        generated_paths.append(data[i])
        
    return generated_paths

In [36]:
paths = generate_random_image_paths(35)
print(paths)

['/kaggle/working/dataset/143.jpg', '/kaggle/working/dataset/387.jpg', '/kaggle/working/dataset/639.jpg', '/kaggle/working/dataset/184.jpg', '/kaggle/working/dataset/21.jpg', '/kaggle/working/dataset/235.jpg', '/kaggle/working/dataset/257.jpg', '/kaggle/working/dataset/94.jpg', '/kaggle/working/dataset/784.jpg', '/kaggle/working/dataset/577.jpg', '/kaggle/working/dataset/482.jpg', '/kaggle/working/dataset/57.jpg', '/kaggle/working/dataset/406.jpg', '/kaggle/working/dataset/449.jpg', '/kaggle/working/dataset/309.jpg', '/kaggle/working/dataset/608.jpg', '/kaggle/working/dataset/879.jpg', '/kaggle/working/dataset/755.jpg', '/kaggle/working/dataset/545.jpg', '/kaggle/working/dataset/228.jpg', '/kaggle/working/dataset/212.jpg', '/kaggle/working/dataset/407.jpg', '/kaggle/working/dataset/347.jpg', '/kaggle/working/dataset/552.jpg', '/kaggle/working/dataset/363.jpg', '/kaggle/working/dataset/441.jpg', '/kaggle/working/dataset/293.jpg', '/kaggle/working/dataset/574.jpg', '/kaggle/working/datas

In [ ]:
results = model(paths, batch=16, device=0)
for result in results:
    result.show()  # display to screen

In [5]:
model = YOLO('/kaggle/working/runs/detect/custom_model3/weights/best.pt')
model.export()

ValueError: Invalid export format='pytorch'. Valid formats are ('torchscript', 'onnx', 'openvino', 'engine', 'coreml', 'saved_model', 'pb', 'tflite', 'edgetpu', 'tfjs', 'paddle', 'ncnn')

In [5]:
import torch
import torchvision

model = torch.jit.load('/kaggle/working/runs/detect/custom_model3/weights/best.torchscript')
model.eval()

im = torchvision.io.read_image('/kaggle/working/dataset/143.jpg')
result = model(im)
print(result)

RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__/ultralytics/nn/tasks.py", line 64, in forward
    model23 = self.model
    _0 = getattr(model23, "0")
    _25 = (_1).forward(act, (_0).forward(act, x, ), )
                             ~~~~~~~~~~~ <--- HERE
    _26 = (_3).forward(act, (_2).forward(act, _25, ), )
    _27 = (_4).forward(act, _26, )
  File "code/__torch__/ultralytics/nn/modules/conv.py", line 12, in forward
    x: Tensor) -> Tensor:
    conv = self.conv
    _0 = (argument_1).forward((conv).forward(x, ), )
                               ~~~~~~~~~~~~~ <--- HERE
    return _0
class Concat(Module):
  File "code/__torch__/torch/nn/modules/conv.py", line 12, in forward
    bias = self.bias
    weight = self.weight
    input = torch._convolution(x, weight, bias, [2, 2], [1, 1], [1, 1], False, [0, 0], 1, False, False, True, True)
            ~~~~~~~~~~~~~~~~~~ <--- 